In [ ]:
# default_exp model

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# export
import torch
import torch.nn as nn

from functools import partial
from fastai import *
from fastai.basic_train import Learner

from keraTorch.data import create_db
from keraTorch.layers import *
from keraTorch.losses import *

from IPython.display import clear_output

In [ ]:
# export
class Sequential:
    def __init__(self, model=None):
        self.layers = []
        self.last_dim = None
        self.model = model
        
    def add(self, layer):
        layer = layer.get_layer(self.last_dim)
        self.last_dim = layer['output_dim']
        self.layers.extend(layer['layers'])
        
    def compile(self, loss, optimizer=None):
        if len(self.layers) > 0:
            self.model = nn.Sequential(*self.layers)
        self.loss = loss
        
    def fit(self, x, y, bs, epochs, lr=1e-3, one_cycle=True, get_lr=True):
        db = create_db(x, y, bs)
        self.learn = Learner(db, self.model, loss_func=self.loss)
        if one_cycle:
            self.learn.fit_one_cycle(epochs, lr)
        else:
            self.learn.fit(epochs, lr)
               
    def lr_find(self, x, y, bs):
        db = create_db(x, y, bs)
#         breakpoint()
        learn = Learner(db, self.model, loss_func=self.loss)
        learn.lr_find()
        clear_output()
        learn.recorder.plot(suggestion=True)
        
    def predict(self, x):
        self.learn.model.eval()
        with torch.no_grad():
            y_preds = self.learn.model(torch.Tensor(x))
        return y_preds.cpu().numpy()

In [ ]:
from sklearn.datasets import load_iris
import numpy as np

data = load_iris()
x, y = data['data'], data['target']
x.shape, y.shape, x.dtype, y.dtype

((150, 4), (150,), dtype('float64'), dtype('int64'))

In [ ]:
model = Sequential()
model.add(Dense(2, x.shape[1], activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(len(data['target_names'])))
model.add(Activation('softmax'))

In [ ]:
model.compile(ce4softmax)

In [ ]:
bs = 50
model.lr_find(x, y, bs=bs)

> <ipython-input-4-ca25c2688976>(29)lr_find()
-> learn = Learner(db, self.model, loss_func=self.loss)
(Pdb) exit


BdbQuit: 

In [ ]:
model.fit(x, y, bs, epochs=3, lr=1e-1)

In [ ]:
preds = model.predict(x[:2])
preds

In [ ]:
preds.sum(axis=-1, keepdims=True)

In [ ]:
# hide
from nbdev.export import *
notebook2script()